# Figures from Water Column Model Runs

In [68]:
#from IPython.core.interactiveshell import InteractiveShell
#InteractiveShell.ast_node_interactivity = 'all'

# Data Science and Visualization Imports
from plotly.subplots import make_subplots
import geopandas as gpd
from pyproj import Proj
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import xarray as xr
import numpy as np

### Data Imports

In [69]:
wc_results_path = 'results_data/wc_runs_all.nc'
baseline_path = 'results_data/baseline_wc_full.nc'
input_data_path = 'results_data/usgs_data_full.csv'
locs_path = 'results_data/locs_data.csv'
wc_ds = xr.open_dataset(wc_results_path)
base_wc_ds = xr.open_dataset(baseline_path)
df = pd.read_csv(input_data_path)
locs = pd.read_csv(locs_path)

In [70]:
base_wc_ds

<xarray.Dataset>
Dimensions:     (Model Runs: 114, index: 80)
Coordinates:
  * index       (index) int32 0 1 2 3 4 5 6 7 8 9 ... 71 72 73 74 75 76 77 78 79
Dimensions without coordinates: Model Runs
Data variables:
    U Velocity  (Model Runs, index) float64 ...
    Q           (Model Runs, index) float64 ...
    Q2          (Model Runs, index) float64 ...
    Q2L         (Model Runs, index) float64 ...
    Z           (Model Runs, index) float64 ...
    Kq          (Model Runs, index) float64 ...
    nu_t        (Model Runs, index) float64 ...
Attributes:
    description:  Model run results for Bay-side sites.

In [71]:
wc_ds

<xarray.Dataset>
Dimensions:     (Model Runs: 114, index: 80)
Coordinates:
  * index       (index) int32 0 1 2 3 4 5 6 7 8 9 ... 71 72 73 74 75 76 77 78 79
Dimensions without coordinates: Model Runs
Data variables:
    U Velocity  (Model Runs, index) float64 ...
    Q           (Model Runs, index) float64 ...
    Q2          (Model Runs, index) float64 ...
    Q2L         (Model Runs, index) float64 ...
    Z           (Model Runs, index) float64 ...
    Kq          (Model Runs, index) float64 ...
    nu_t        (Model Runs, index) float64 ...
Attributes:
    description:  Model run results for Bay-side sites.

In [72]:
df = (pd.merge(df, locs, how='left', on='plot_ID'))
df = df.drop(['Unnamed: 0_x', 'Unnamed: 0_y'], axis=1)

In [73]:
p2 = Proj(init="epsg:7131", proj="utm", zone=10)
df['Lon'], df['Lat'] = p2(df['Easting'],df['Northing'],inverse=True)
#df = df.drop(['Easting', 'Northing'], axis=1)
df.head()

c:\Users\erich\anaconda3\envs\EFMHModeling_20220915\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6



,Season,date,site,plot_ID,Elevation,channel_dist,ave_ht,cover,vol_est,vol_calc,...,frontal_20,frontal_25,frontal_30,wc_height,Easting,Northing,Transect,Distance,Lon,Lat
0,Summer,7/28/2021,Bay,A0.5,2.1313,0.5,6.5,0.10,6500,233.351057,...,0.000000,0.0,0.0,0.6759,575481.457229,4.159935e+06,A,0.5,-122.145098,37.583378
1,Summer,7/28/2021,Bay,A2,2.3217,2.0,11.0,0.45,49500,1777.058049,...,0.000000,0.0,0.0,0.4855,575483.018742,4.159936e+06,A,2.0,-122.145080,37.583388
2,Summer,7/28/2021,Bay,A6,2.2421,6.0,19.0,0.55,104500,3751.566993,...,3734.753837,0.0,0.0,0.5651,575486.683492,4.159938e+06,A,6.0,-122.145039,37.583404
3,Summer,7/28/2021,Bay,A12,2.2417,12.0,15.0,0.75,112500,4038.768294,...,0.000000,0.0,0.0,0.5655,575492.070254,4.159941e+06,A,12.0,-122.144977,37.583428
4,Summer,7/28/2021,Bay,A24,2.2217,24.0,11.0,0.55,60500,2171.959838,...,0.000000,0.0,0.0,0.5855,575502.472088,4.159947e+06,A,24.0,-122.144859,37.583481


Compiling Data from modeling runs for analysis

In [74]:
a = len(wc_ds['U Velocity'])
df1 = pd.DataFrame({
    'plot_ID': df['plot_ID'],
    'mean_u': ([vel.values.mean() for vel in wc_ds['U Velocity']]),
    'mean_u_25': ([vel[:20].values.mean() for vel in wc_ds['U Velocity']]),
    'mean_u_50': ([vel[:40].values.mean() for vel in wc_ds['U Velocity']]),
    'mean_u_75': ([vel[:60].values.mean() for vel in wc_ds['U Velocity']]),
    'Q2': ([val.values.max() for val in wc_ds['Q2']]),
    'Hveg/WC': df['ave_ht']/df['wc_height'],
    'wc_height': df['wc_height'],
    'avg_ht': df['ave_ht'],
    'density_total': df['density_final'],
    'frontal_total': df['frontal_total'],
    'site': df['site'],
    'lon': df['Lon'],
    'lat': df['Lat'],
    'run': np.repeat('Vegetated Run',a)
})
df2 = pd.DataFrame({
    'plot_ID': df['plot_ID'],
    'mean_u': ([vel.values.mean() for vel in base_wc_ds['U Velocity']]),
    'mean_u_25': ([vel[:20].values.mean() for vel in base_wc_ds['U Velocity']]),
    'mean_u_50': ([vel[:40].values.mean() for vel in base_wc_ds['U Velocity']]),
    'mean_u_75': ([vel[:60].values.mean() for vel in base_wc_ds['U Velocity']]),
    'Q2': ([val.values.max() for val in base_wc_ds['Q2']]),
    'Hveg/WC': df['ave_ht']/df['wc_height'],
    'wc_height': df['wc_height'],
    'avg_ht': df['ave_ht'],
    'density_total': df['density_final'],
    'frontal_total': df['frontal_total'],
    'site': df['site'],
    'lon': df['Lon'],
    'lat': df['Lat'],
    'run': np.repeat('Baseline Run',a)
})
dff = pd.concat([df1, df2])
dff.sample(5)

,plot_ID,mean_u,mean_u_25,mean_u_50,mean_u_75,Q2,Hveg/WC,wc_height,avg_ht,density_total,frontal_total,site,lon,lat,run
94,I24,0.535252,0.002339,0.167641,0.364292,0.168553,25.278810,0.6725,17.0,5359.089301,24530.542248,Creek,-122.142195,37.583931,Vegetated Run
52,H2,1.246928,0.289861,0.721031,1.021061,0.236286,10.327530,0.6778,7.0,4413.367659,8318.315364,Interior,-122.144146,37.588105,Vegetated Run
41,F6,0.522126,0.001461,0.169997,0.359754,0.148074,24.034335,0.5825,14.0,4413.367659,16636.630728,Interior,-122.143055,37.587382,Vegetated Run
67,B12,11.710292,10.570477,11.138013,11.472735,0.808482,19.027850,0.5781,11.0,5989.570395,17740.080726,Bay,-122.144724,37.583123,Baseline Run
4,A24,0.698899,0.038417,0.298809,0.522201,0.151452,18.787361,0.5855,11.0,3467.646018,10270.573052,Bay,-122.144859,37.583481,Vegetated Run


Dropping invalid location values from dataframe

In [75]:
dff = dff.replace([np.inf, -np.inf], np.nan)
dff = dff.dropna()

## EDA and Scatterplots

In [76]:
fig = px.scatter(dff, x='Q2', y='avg_ht', color='site', symbol='run', trendline='ols', template='plotly_dark')
fig.update_layout(
    height=500,
    autosize=True,)
fig.show()

In [77]:
fig = px.scatter(dff, x='mean_u', y='wc_height', trendline='ols', symbol='run', color='site', template='plotly_dark')
fig.update_layout(
    height=500,
    autosize=True,)
fig.show()

In [78]:
fig = px.scatter(dff, x='mean_u_25', y='density_total', symbol='run', color='site', trendline='ols', template='plotly_dark')
#fig = px.scatter(base_df, x='mean_u_25', y='density_total', symbol='site', color='site')
fig.update_layout(
    height=500,
    autosize=True,)
fig.show()

In [79]:
fig = px.scatter(dff, x='mean_u', y='Hveg/WC', symbol='run', color='site', trendline='ols', template='plotly_dark')
#fig = px.scatter(base_df, x='mean_u', y='Hveg/WC', symbol='site', color='site')
fig.update_layout(
    height=500,
    autosize=True,)
fig.show()

## Contour Plots

In [80]:
fig = make_subplots(rows=1, cols=2,
                    subplot_titles=('Baseline Runs', 'Vegetated Runs'))

cbarlocs = [.85, .5, .15]

fig.add_trace(go.Contour(
        z=df2['mean_u_25'],
        x=df2['density_total'], # horizontal axis
        y=df2['Hveg/WC'],# vertical axis,
        colorbar_x=0.46,
        line_smoothing=0.85), 1, 1)
fig.add_trace(go.Contour(
        z=df1['mean_u_25'],
        x=df1['density_total'], # horizontal axis
        y=df1['Hveg/WC'],# vertical axis,
        line_smoothing=0.85), 1, 2)

#fig = go.Figure(data =
 #   go.Contour(
  #      z=dff['mean_u_25'],
   #     x=dff['density_total'], # horizontal axis
    #    y=dff['Hveg/WC'],# vertical axis,
     #))
fig.update_layout(
    title="Velocity Contour Plot",
    xaxis_title="Density",
    yaxis_title="Hveg/WC",
    width=1400,
    height=700,
    autosize=True,
    template='plotly_dark')
fig.show()

## Maps

In [91]:
fig = px.scatter_mapbox(dff[dff['run']=='Vegetated Run'], 
    lat="lat", 
    lon="lon", 
    color='mean_u', 
    size='Hveg/WC',
    size_max=12,
    hover_name="plot_ID", 
    hover_data=["site", "mean_u", "density_total", "Hveg/WC"], 
    zoom=15, 
    height=600)
    
fig.update_layout(
    mapbox_style="white-bg",
    mapbox_layers=[
        {
            "below": 'traces',
            "sourcetype": "raster",
            "sourceattribution": "United States Geological Survey",
            "source": [
                "https://basemap.nationalmap.gov/arcgis/rest/services/USGSImageryOnly/MapServer/tile/{z}/{y}/{x}"
            ]
        }
      ])
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0},template='plotly_dark')
fig.show()

## Animations